In [1]:
import re
from simhash import Simhash, SimhashIndex

def get_features(s):
    width = 3
    s = s.lower()
    s = re.sub(r'[^\w]+', '', s)
    return [s[i:i + width] for i in range(max(len(s) - width + 1, 1))]


data = {
    1: 'How are you? I am fine. blar blar. Thanks.',
    2: 'How are you i am fine. blar blar. thank',
    3: 'this is simhash test.',
}

objs = [(str(k), Simhash(get_features(v))) for k, v in data.items()]
index = SimhashIndex(objs, k=3)

In [7]:
print(
    "Number of bits in fingerprint:", index.f, "\n",
    "Hamming distance threshold:", index.k, "\n",
    "Distance values up to threshold:", [i for i in range(index.k + 1)], "\n",
    "Indices of the first bits in each bin, such that enough bins are created to meet k criteria:", index.offsets
)

Number of bits in fingerprint: 64 
 Hamming distance threshold: 3 
 Distance values up to threshold: [0, 1, 2, 3] 
 Indices of the first bits in each bin, such that enough bins are created to meet k criteria: [0, 16, 32, 48]


Cell above shows how Google optimized the bit search by creating bins of bits.  Instead of sorting the raw hashes (and shifting the raw hashes 1 bit at a time), they used the distance threshold, k, to determine how many bins should be used.  This is what they mean by "precomputing F' such that some existing fingerprint, F, is at most Hamming distance k away from F'".  They create the permutations using bit bins instead of the full table of bit permutations.  

In [27]:
offsets = [(i, offset) for i, offset in enumerate(index.offsets)]
ms_if_true = [2**(index.f - o[1]) - 1 for o in offsets]
ms_if_false = [2**(offsets[o[0]+1][1] - o[1]) - 1 for o in offsets if o[0] != 3]
print(
    "SimHash value:", objs[0][1].value, "\n",
    "Offsets tuple:", offsets, "\n",
    "Len offsets -1 (the if condition in get_keys):", len(index.offsets) - 1, "\n",
    "All values of m calculated by get_offsets if contition True:", ms_if_true, "\n",
    "All values of m calculated by get_offsets if condition False:", ms_if_false, "\n",
    "Shifted simhash for first offset:", objs[0][1].value >> offsets[0][1], "\n",
    "Shifted simhash for first offset:", offsets[0][1] & ms_if_false[0], "\n",
    "Shifted simhash for first offset with m:", objs[0][1].value >> offsets[0][1] & ms_if_false[0], "\n",
    [k for k in index.get_keys(objs[0][1])]
)

SimHash value: 7604582634754244165 
 Offsets tuple: [(0, 0), (1, 16), (2, 32), (3, 48)] 
 Len offsets -1 (the if condition in get_keys): 3 
 All values of m calculated by get_offsets if contition True: [18446744073709551615, 281474976710655, 4294967295, 65535] 
 All values of m calculated by get_offsets if condition False: [65535, 65535, 65535] 
 Shifted simhash for first offset: 7604582634754244165 
 Shifted simhash for first offset: 0 
 Shifted simhash for first offset with m: 32325 
 ['7e45:0', '91a1:1', 'e79d:2', '6988:3']


So get_keys gets the permuted fingerprint keys, and index.bucket holds the permuted fingerprints table.  This table is a dict of {fingerprint: (hash, obj_id)}.  

Given fingerprint F (and integer k defined in index.k), identify all permuted fingerprints in T (index.bucket) whose top bit position match the top bit positions of pi(F), the permutated F.  

In [37]:
for simhash_key, hashes in index.bucket.items():
    print(
        "Fingerprint F' key:", simhash_key, "\n",
        "Nbr Items:", len(hashes), "\n",
        "Hash Values:", [hash_details.split(",")[0] for hash_details in hashes], "\n",
        "Doc Ids:", [int(hash_details.split(",")[1]) for hash_details in hashes], "\n",
    )
    if simhash_key[-1:] == "3":
        print(".......New Bucket....... \n")

Fingerprint F' key: 7e45:0 
 Nbr Items: 2 
 Hash Values: ['6988e79d91a17e45', '69cae79d91e37e45'] 
 Doc Ids: [1, 2] 

Fingerprint F' key: 91a1:1 
 Nbr Items: 1 
 Hash Values: ['6988e79d91a17e45'] 
 Doc Ids: [1] 

Fingerprint F' key: e79d:2 
 Nbr Items: 2 
 Hash Values: ['6988e79d91a17e45', '69cae79d91e37e45'] 
 Doc Ids: [1, 2] 

Fingerprint F' key: 6988:3 
 Nbr Items: 1 
 Hash Values: ['6988e79d91a17e45'] 
 Doc Ids: [1] 

.......New Bucket....... 

Fingerprint F' key: 91e3:1 
 Nbr Items: 1 
 Hash Values: ['69cae79d91e37e45'] 
 Doc Ids: [2] 

Fingerprint F' key: 69ca:3 
 Nbr Items: 1 
 Hash Values: ['69cae79d91e37e45'] 
 Doc Ids: [2] 

.......New Bucket....... 

Fingerprint F' key: b1b0:0 
 Nbr Items: 1 
 Hash Values: ['25961fbb1e26b1b0'] 
 Doc Ids: [3] 

Fingerprint F' key: 1e26:1 
 Nbr Items: 1 
 Hash Values: ['25961fbb1e26b1b0'] 
 Doc Ids: [3] 

Fingerprint F' key: 1fbb:2 
 Nbr Items: 1 
 Hash Values: ['25961fbb1e26b1b0'] 
 Doc Ids: [3] 

Fingerprint F' key: 2596:3 
 Nbr Items: 1 
 H

get_near_dups takes the candidates identified from get_keys, and determines which of them are closer than k.  These are the near duplicates.  

In [42]:
print(objs[0][1].distance(objs[1][1]), objs[0][1].distance(objs[2][1]), objs[1][1].distance(objs[2][1]), objs[1][1].distance(objs[0][1]))

4 36 36 4
